In [ ]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import pandas as pd
from pandas import DataFrame

### Load and format the data

In [ ]:
parameterDF = pd.read_csv("ParameterData.csv", header=0, index_col=0, sep=',')
gravity, angle, stretch, length, mass, spring, success = parameterDF.values 

# Also group the successes and failures for plotting later
groups = parameterDF.T.groupby( parameterDF.T.columns[-1] ) # group by the last parameter (performance metric)
successGroup = groups.get_group(1)
failureGroup = groups.get_group(0)

### Prepare the KNN classifier and plots

In [ ]:
k = (int)( sqrt(parameterDF.shape[1]) ) # k nearest neighbors
h = 0.005  # step size in the mesh
cmap = ListedColormap( ['#F08080', '#98FB98'] ) # colors for prediction mesh
params = parameterDF.T.columns.tolist()[:-1] # Use dataframe labels for plot labels

In [ ]:
fig, axarr = plt.subplots( len(params), len(params), figsize=(16,16) )
classifiers = []

for rowNum in range( len(params) ):
    
    for colNum in range( len(params) ):
        
        # Get training data for fitting KNN
        x1 = parameterDF.T[ params[colNum] ] # x-axis in plot
        x2 = parameterDF.T[ params[rowNum] ] # y-axis in plot
        X = np.column_stack( (x1, x2) )
        classifier = neighbors.KNeighborsClassifier( k, weights='distance' ).fit( X, success )
        classifiers.append(classifier)
    
        # Plot the decision boundary, assigning a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        xMin, xMax = x1.min(), x1.max()
        yMin, yMax = x2.min(), x2.max()
        xx, yy = np.meshgrid(np.arange(xMin-1, xMax+1, h),
                             np.arange(yMin-1, yMax+1, h))
        Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)       
        axarr[rowNum,colNum].pcolormesh(xx, yy, Z, cmap=cmap)
    
        # Plot the training data
        xSuccesses = successGroup[ params[colNum] ] 
        ySuccesses = successGroup[ params[rowNum] ]
        axarr[rowNum,colNum].scatter( xSuccesses, ySuccesses, marker='o', color='#006400', s=70 )
    
        xFailures = failureGroup[ params[colNum] ]
        yFailures = failureGroup[ params[rowNum] ]
        axarr[rowNum,colNum].scatter( xFailures, yFailures, marker='s', color='#FF0000', s=50 )

        axarr[rowNum,colNum].set_xlim( xMin-0.01, xMax+0.01 )
        axarr[rowNum,colNum].set_ylim( yMin-0.01, yMax+0.01 )

# Fine tune the plots   
topPlots = axarr[0, :]
bottomPlots = axarr[-1, :]  
leftPlots = axarr[:, 0]
rightPlots = axarr[:, -1]

for count in range( len(params) ):
    # X-axes
    topPlots[count].xaxis.tick_top() 
    topPlots[count].xaxis.set_label_position( 'top' ) 
    topPlots[count].set_xlabel( params[count] ) 
    bottomPlots[count].set_xlabel( params[count] )  
    if count != 0 and count != len(params)-1: # for the middle plots
        plt.setp( [ ax.get_xticklabels() for ax in axarr[count, :] ], visible=False )
    # Y-axes
    rightPlots[count].yaxis.tick_right()
    rightPlots[count].yaxis.set_label_position( 'right' )
    rightPlots[count].set_ylabel( params[count] )
    leftPlots[count].set_ylabel( params[count] )
    if count != 0 and count != len(params)-1: # for the middle plots
        plt.setp( [ ax.get_yticklabels() for ax in axarr[:, count] ], visible=False )

plt.setp( [ ax.get_xticklabels() for ax in axarr[0, :] ], rotation=45 ) 
plt.setp( [ ax.get_xticklabels() for ax in axarr[-1, :] ], rotation=45 )
plt.subplots_adjust( hspace=0, wspace=0 )

plt.savefig('KNNPlot.png')